In [1]:
!pip install pymongo
!pip install lxml 


     |████████████████████████████████| 530 kB 786 kB/s eta 0:00:01
     |████████████████████████████████| 5.4 MB 3.4 MB/s eta 0:00:01


In [3]:
import pymongo
import json
import os
import re, time, requests
from lxml import etree
from multiprocessing import Pool
from datetime import date
import os
import datetime

In [70]:
def make_params(sceng= 0,area = '',min = '',max= ''):
    paramslist = []
    my_params = {'scneg': sceng,  # 搜尋參數
                 'area': area,  # 指定區域
                 'scmin': min,
                 'scmax': max
                 }
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36',
        'Referer': 'https://www.104.com.tw/job/'}
    searchpageurl = 'https://www.104.com.tw/jobs/search/?'
    ss = requests.session()

    res = ss.get(url=searchpageurl, headers=headers, params=my_params)
    html = etree.HTML(res.text)
    try:
        totalpage = int(html.xpath('//script[contains(text(),"totalPage")]/text()')[0].split('totalPage":')[-1].split(
            ',"totalCount"')[0])  # 取得js中的total page
        print("totalPage:", totalpage)
    except Exception as e:
        print(e)
        totalpage = 1
    for page in range(1,totalpage+1):
        my_params = {'scneg': sceng,  # 搜尋參數
                     'area': area,  # 指定區域
                     'scmin': min,
                     'scmax': max,
                     'page': page
                     }
        paramslist.append(my_params)
    return paramslist

#帶入myparams參數取得後五碼的list
def getindex(my_params):
    joburl = []
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.4147.135 Safari/537.36',
        'Referer': 'https://www.104.com.tw/job/'}
    searchpageurl = 'https://www.104.com.tw/jobs/search/?'
    ss = requests.session()

    try:
        res = ss.post(url=searchpageurl, headers=headers, params=my_params)#allow_redirects=False
    except Exception as e:
        print(e,"continuing...")
    try:
        if res.status_code == 200:
            html = etree.HTML(res.text)
            for ajax_content in html.xpath('//a[contains(@class,"js-job-link")]/@href'):
                joburl.append(ajax_content.split('?')[0].split('/')[-1])
            return joburl  #list

        else :
            print("wrong code :",res.status_code)
    except Exception as exc:
        print(exc)
#把抓下來的後五碼存進json檔
def dump_json_file(query_dict):
    dumped_json_cache = json.dumps(query_dict)
    filename = date.today().strftime("%Y-%m-%d")
    fw = open('./{}.json'.format(filename), "w")
    fw.write(dumped_json_cache)
    fw.close()
    print('dump the data successfully')

def crowl(url):
    tmpdict = {}
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36',
    'Referer': 'https://www.104.com.tw/job/'}

    ss=requests.session()
    res = ss.get(url= url , headers = headers)
    if res.status_code == 200:
        try:
            return res.json()
        except json.decoder.JSONDecodeError:
            print(res.json)

    else:
        print('請求網頁json錯誤, 錯誤狀態碼：', res.status_code)

#資料清洗、重新排版
def extract(joburl,data_dict):
    job_dict = {joburl: {}}
    #print('PID:{} , task:{}'.format(os.getpid(),joburl))
    try:
        data_dict = data_dict['data']

        # job title
        job_dict[joburl]['jobName'] = data_dict['header']['jobName']
        job_dict[joburl]['appearDate'] = data_dict['header']['appearDate']

        # company detail
        job_dict[joburl]['companyName'] = data_dict['header']['custName']
        job_dict[joburl]['companyUrl'] = data_dict['header']['custUrl']
        job_dict[joburl]['industry'] = data_dict['industry']
        job_dict[joburl]['addressRegion'] = data_dict['jobDetail']['addressRegion']
        job_dict[joburl]['longitude'] = data_dict['jobDetail']['longitude']
        job_dict[joburl]['latitude'] = data_dict['jobDetail']['latitude']

        # condition
        job_dict[joburl]['acceptRole'] = data_dict['condition']['acceptRole']
        job_dict[joburl]['workExp'] = data_dict['condition']['workExp']
        job_dict[joburl]['edu'] = data_dict['condition']['edu']
        job_dict[joburl]['major'] = data_dict['condition']['major']
        job_dict[joburl]['language'] = data_dict['condition']['language']
        job_dict[joburl]['skill'] = data_dict['condition']['skill']
        job_dict[joburl]['certificate'] = data_dict['condition']['certificate']
        job_dict[joburl]['other'] = data_dict['condition']['other']

        # job Detail
        job_dict[joburl]['jobDescription'] = data_dict['jobDetail']['jobDescription']
        job_dict[joburl]['jobCategory'] = data_dict['jobDetail']['jobCategory']
        job_dict[joburl]['jobType'] = data_dict['jobDetail']['jobType']
        job_dict[joburl]['manageResp'] = data_dict['jobDetail']['manageResp']
        job_dict[joburl]['businessTrip'] = data_dict['jobDetail']['businessTrip']
        job_dict[joburl]['workPeriod'] = data_dict['jobDetail']['workPeriod']
        job_dict[joburl]['vacationPolicy'] = data_dict['jobDetail']['vacationPolicy']
        job_dict[joburl]['startWorkingDay'] = data_dict['jobDetail']['startWorkingDay']
        job_dict[joburl]['needEmp'] = data_dict['jobDetail']['needEmp']

        # salary
        job_dict[joburl]['salary'] = data_dict['jobDetail']['salary']
        job_dict[joburl]['salaryMin'] = data_dict['jobDetail']['salaryMin']
        job_dict[joburl]['salaryMax'] = data_dict['jobDetail']['salaryMax']
        job_dict[joburl]['salaryType'] = data_dict['jobDetail']['salaryType']
        job_dict[joburl]['welfare'] = data_dict['welfare']['welfare']

        return job_dict
    except Exception as e:
        print(e)
        print(joburl)
        return job_dict

def write_json(filename,tmpdict):
    #pprint.pprint(tmpdict)
    with open('./'+filename,'w') as f:
        f.write(json.dumps(tmpdict))
    print("write{} successfully".format(filename))

def open_json_file(CACHE_FNAME):
    try:
        cache_file = open(CACHE_FNAME, 'r')
        cache_contents = cache_file.read()
        CACHE_DICTION = json.loads(cache_contents)
        cache_file.close()
        return CACHE_DICTION

    except Exception as e:
        print("no any cache:",e)
        CACHE_DICTION = {}
        return CACHE_DICTION

# 跟mongodb建立連線
def mongo_connect_build():
    global mycol
    client = pymongo.MongoClient(host="mongodb", port=27017)
    # client = pymongo.MongoClient(host="mongodb", port=27017)

    # 選擇使用的db,不存在則會在資料輸入時自動建立
    db = client['Topic_104']

    # 選擇collection,不存在則會在資料輸入時自動建立
    mycol = db["Jobs"]


def getlist():
    if not os.path.exists('./data/'):
        os.mkdir('./data/')
    else:
        pass
    try:
        l = catch()['index']
    except TypeError:
        print("doesn't need match...........")
        l=[]
    print("please wait...")
    for i in os.listdir('./data/'):
        if i.strip('.json') in l:
            #del list[list.index(i.strip('.json'))]
            l.remove(i.strip('.json'))
    print("got catching list!!",l)
    return l


def catch():

    cachefilename = './{}.json'.format(date.today().strftime("%Y-%m-%d"))  #檔案名稱為當天日期
    try:
        with open(cachefilename,'r') as f:
            cache_contents = f.read()
            cachedict = json.loads(cache_contents)
        return cachedict
    except FileNotFoundError :
        print("---------no cache file-------------")
        return None

def cach_to_json(index):
    if len(index) < 10 :
        url = "https://www.104.com.tw/job/ajax/content/"+index
        try:
            j = extract(index,crowl(url))
            write_json('./data/{}.json'.format(index), j)
            print("write:{}".format(index))
        except Exception as e:
            print("error:",e)
            time.sleep(2)
    else:
        pass


def main2():
    arealist = [str(i) for i in range(6001001001, 6001001013)]  # 台北市區域
    for i in range(6001002001, 6001002030):  # 新北市區域
        arealist.append(str(i))
    arealist=[6001002001,6001002002]
    paramlist = []
    print("paramslist making ")
    for area in arealist:
        paramlist += make_params(sceng=0, area=area, min=0, max=37999) # params(salary0~37999,所有地區)
        #paramlist += make_params(sceng=1, area=area, min=38000, max=40000)
        #paramlist += crowler104.make_params(sceng=0, area=area, min=40001, max='')
    print("paramslist done",paramlist)
    print("paramslist length:",len(paramlist))
    print("getting index.....")

    index = []

    for i in paramlist:
        # try:
        print(i)
        print(getindex(i))
        index += getindex(i)
        # except :
        #     #print("errors",i)

    indexjson = {'index': index} if index != [] or len(index)<10 else False
    print('all done,total index amount:{}'.format(len(indexjson['index'])))
    dump_json_file(indexjson)

# 輸入資料
def data_insert(CACHE_DICTION):
    try:
        mycol.insert_one(CACHE_DICTION)
        print("insert successfully!：",CACHE_DICTION.keys())
    except Exception as e:
        print("insert error:",e)

In [75]:
if __name__ == "__main__":

    main2()

    list2 = getlist()
    for i in list2:
        cach_to_json(i)



    mongo_connect_build() # 連線mongodb
    # 班代設計程式爬下來之後,都會在data資料夾裡面,將各個檔案輸入到mongodb
    for CACHE_FNAME in os.listdir('./data/'):
        CACHE_DICTION = open_json_file('./data/'+CACHE_FNAME)
        #print(CACHE_DICTION.keys())
        
        data_insert(CACHE_DICTION)


    for i in mycol.find({},{"_id":0}).limit(10):
        print("mongo top 10",str(i.values()))
    print(datetime.datetime.utcnow())

paramslist making 
totalPage: 2
totalPage: 3
paramslist done [{'scneg': 0, 'area': 6001002001, 'scmin': 0, 'scmax': 37999, 'page': 1}, {'scneg': 0, 'area': 6001002001, 'scmin': 0, 'scmax': 37999, 'page': 2}, {'scneg': 0, 'area': 6001002002, 'scmin': 0, 'scmax': 37999, 'page': 1}, {'scneg': 0, 'area': 6001002002, 'scmin': 0, 'scmax': 37999, 'page': 2}, {'scneg': 0, 'area': 6001002002, 'scmin': 0, 'scmax': 37999, 'page': 3}]
paramslist length: 5
getting index.....
{'scneg': 0, 'area': 6001002001, 'scmin': 0, 'scmax': 37999, 'page': 1}
['2klss', '6ovtl', '7166q', '71gnf', '6yqxh', '7211b', '6yqmc', '6yqmr', '71z7n', '70t12', '1xyls', '70wqh', '71yrh', '70szy', '67hnu', '6huiq', '70m1x', '70ay3', '6zshe', '6zyiq']
{'scneg': 0, 'area': 6001002001, 'scmin': 0, 'scmax': 37999, 'page': 2}
['71ah4', '70ayl', '71wrb', '71w5i', 'w7ec', '4fm9j', '70fld', '3vkzz', '3yozs', '6obol', '60yld', '60yxe', '6mqh9', '6y7vr', '60yw1', '60yy1', '6mqh6', '6lfxv', '6wdz2']
{'scneg': 0, 'area': 6001002002, 'scm